# Convolutional Neural Network

### Tutorial: https://pythonprogramming.net/convolutional-neural-network-kats-vs-dogs-machine-learning-tutorial/
### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

#### Build the CNN Model

##### Architecture:
- Step 1 - Convolution2D + MaxPooling layer
    - Arguments: input_shape (64,64,3), 32 filters, 3x3 kernel, relu activation
- Step 2.1-2.n - Adding new "n" Convolution2D + MaxPooling layer (more layers = more parameters = more complexity - care overfit)
- Step 3 - Flattening
- Step 4.1 - 4.n - Dense Layer
    - Arguments: 128 units, relu activation
- Step 5 - Output Layer (Dense Layer with units = 1, sigmoid activation)

In [2]:
classifier = Sequential()

# Step 1 - Convolution + MaxPooling layer
# Arguments: 32 filters, 3x3 kernel, relu activation
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 2 - Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

#### Compiling the CNN
- optimizer = 'adam'
- loss = 'binary_crossentropy' - binary problem (either cat or dog)
- metrics = ['accuracy', ...]

In [3]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images
#### For Image Augmentation, verify https://keras.io/preprocessing/image/ - ImageDataGenerator class
#### Alternative: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
#### Or do it manually as in previous exercises

- Step 1 - ImageDataGenerator class for data train and test data generation
    - Arguments: Rescale, shear_range, zoom_range, horizontal_flip
    - For test dataset, only rescale is required

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

- Step 2 - ImageDataGenerator class - use the "flow_from_directory" function to read the input dataset and prepare for augmentation
    - Arguments: directory, target_size, batch_size, class_mode='binary'

In [5]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


- Step 3 - fit the model using "fit_generator" function
    - Arguments: training_dataset, steps_per_epoch, epochs, validation_data, validation_steps
    
- NOTE: install pillow package using "pip install pillow" or "conda install pillow"

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from datetime import datetime

es = EarlyStopping(monitor='val_loss', patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = TensorBoard(log_dir = logs,
                          histogram_freq = 1,
                          profile_batch = '500,520')

classifier.fit_generator(training_set,
                         steps_per_epoch = 250, # 8000 cases / 32 cases per batch (batch_size) = 250
                         epochs = 25, # play with number of epochs and benchmark results
                         callbacks = [es, mc, tensorboard],
                         validation_data = test_set,
                         validation_steps = 62, # 2000 cases / 32 cases per batch (batch_size) = 62.5
                         workers = 4) 

C:\Users\filas\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
250/250 [==============================] - 95s 359ms/step - loss: 0.6911 - accuracy: 0.5521 - val_loss: 0.5938 - val_accuracy: 0.7102
Epoch 2/25
250/250 [==============================] - 10s 38ms/step - loss: 0.6035 - accuracy: 0.6806 - val_loss: 0.5873 - val_accuracy: 0.6915
Epoch 3/25
250/250 [==============================] - 10s 40ms/step - loss: 0.5876 - accuracy: 0.6845 - val_loss: 0.5810 - val_accuracy: 0.7112
Epoch 4/25
250/250 [==============================] - 10s 39ms/step - loss: 0.5403 - accuracy: 0.7329 - val_loss: 0.5717 - val_accuracy: 0.7213
Epoch 5/25
250/250 [==============================] - 10s 39ms/step - loss: 0.5234 - accuracy: 0.7334 - val_loss: 0.5000 - val_accuracy: 0.7641
Epoch 6/25
250/250 [==============================] - 10s 39ms/step - loss: 0.4817 - accuracy: 0.7660 - val_loss: 0.4901 - val_accuracy: 0.7757
Epoch 7/25
250/250 [==============================] - 10s 38ms/step - loss: 0.4675 - accuracy: 0.7823 - val_loss: 0.4877 - val_accuracy

### Part 3 - Making new model predictions for human validation
#### Using the trained model, use images located in dataset/single_prediction folder to verify it's performance

In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)